In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
df = pd.read_csv('df_M30.csv', delimiter=';')

In [ ]:
new_df = df[df['oldID'].str.contains('PM1.*1$')]

In [ ]:
new_df = new_df.loc[new_df['oldID'].str.len() == 7]

In [ ]:
new_df['KM'] = new_df["oldID"].str[3:5].astype(str) + "." + new_df["oldID"].str[5:6].astype(str)

In [ ]:
new_df = new_df[new_df['fecha'].str.contains('2015-08-04')]

In [ ]:
new_df[['KM']] = new_df.loc[:,['KM']].astype(np.float64)

In [ ]:
new_df.loc[:, 'densidad'] = new_df.loc[:, "intensidad"] / new_df.loc[:, "vmed"]

In [ ]:
new_df.dropna(subset = ['densidad'])

In [ ]:
new_df['Hora'] = new_df["fecha"].str[11:13].astype(str) + "." + new_df["fecha"].str[14:16].astype(str)
new_df[['Hora']] = new_df.loc[:,['Hora']].astype(np.float64)

In [ ]:
new_df.sort_values('Hora').reset_index(drop=True)

In [ ]:
new_df.plot.scatter(x="densidad", y="Hora")

In [ ]:
display(new_df)

In [ ]:
new_df.dtypes

In [ ]:
new_df.to_csv('df_M30_KM_dia.csv', sep=';')

# Database of one day

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
# Leo el CSV y dibujo los scatter

df_un_dia = pd.read_csv('df_M30_KM_dia.csv', delimiter=';')
df_un_dia.plot.scatter(x="densidad", y="Hora")
df_un_dia.plot.scatter(x="Hora", y="KM")

In [ ]:
# Ordeno el dataframe por KM y Horas y quito las mediciones erróneas

df_prueba = df_un_dia.copy()
df_prueba = df_prueba.sort_values(by=['KM', 'Hora'])
df_prueba = df_prueba.reset_index(drop=True)
df_prueba = df_prueba[['ID', 'oldID', 'fecha', 'intensidad', 'ocupacion', 'vmed', 'periodo_integracion',
                       'KM', 'densidad', 'Hora']]
df_prueba = df_prueba[~(df_prueba['KM'] == 0.2)]
df_prueba = df_prueba[~(df_prueba['KM'] == 0.9)]
df_prueba.index = range(len(df_prueba.index))

In [ ]:
# Genero un dataframe nuevo para las interpolaciones (15 minutos -> 1 minuto)

nans = np.where(np.empty_like(np.arange(576000).reshape(57600, 10)), np.nan, np.nan)
df_new = pd.DataFrame(nans, columns=['ID','oldID','fecha','intensidad','ocupacion','vmed','periodo_integracion', 
                                     'KM', 'densidad', 'Hora'])

In [ ]:
# Asigno los valores registrados cada 15 columnas y el resto como NaN para interpolar.

j = 0
for i in df_new.index.values.astype(int):
    print("i: ", i, "j: ", j)
    if (i%15) == 0:
        df_new.loc[i] = df_prueba.values[j]
        if i != 0:
            j=j+1
    else:
        df_new.loc[[i], ['ID']] = df_prueba.loc[[j], ['ID']]
        df_new.loc[[i], ['oldID']] = df_prueba.loc[[j], ['oldID']]
        df_new.loc[[i], ['fecha']] = df_prueba.loc[[j], ['fecha']]
        df_new.loc[[i], ['periodo_integracion']] = 1
        df_new.loc[[i], ['KM']] = df_prueba.loc[[j], ['KM']]

In [ ]:
# Exporto a CSV el DataFrame

df_new.to_csv('df_M30_KM_dia_perinterpol.csv', sep=';')

# Interpolado

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
df_new = pd.read_csv('df_M30_KM_dia_perinterpol.csv', delimiter=';')
#df_new.plot.scatter(x="Hora", y="densidad")
#df_new.plot.scatter(x="Hora", y="KM", c="vmed", colormap='viridis')

df_new = df_new.sort_values(by=['KM', 'Hora'])
df_new = df_new.reset_index(drop=True)
df_new = df_new[['ID', 'oldID', 'fecha', 'intensidad', 'ocupacion', 'vmed', 'periodo_integracion',
                       'KM', 'densidad', 'Hora']]
df_new = df_new[~(df_new['KM'] == 0.2)]
df_new = df_new[~(df_new['KM'] == 0.9)]
df_new.index = range(len(df_new.index))

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(df_new.KM, df_new.Hora, df_new.vmed, c=df_new.vmed, cmap='magma');

#Axes3D.scatter(xs=df_new.Hora, ys=df_new.KM, zs=df_new.vmed, zdir='z', **kwargs)

# Testing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import math
import decimal
%matplotlib inline

In [2]:
# Interpolo.

df = pd.read_csv('df_M30_KM_dia.csv', delimiter=',')

df = df.sort_values(by=['KM', 'Hora'])
df = df.reset_index(drop=True)
df = df[['ID', 'oldID', 'fecha', 'intensidad', 'ocupacion', 'vmed', 'periodo_integracion',
                       'KM', 'densidad', 'Hora']]
df = df[~(df['KM'] == 0.2)]
df = df[~(df['KM'] == 0.9)]
df.index = range(len(df.index))

df_temp = pd.DataFrame([])
df_int = pd.DataFrame([])

for id in np.sort(df.oldID.value_counts().keys()):
    print("oldID: ", id)
    df_aux = df.loc[df['oldID'] == id]
    # Se crea la matriz de interpolación de tamaño: filas del id obtenido * 15 minutos interpolados
    #- 14 últimos minutos del día sin medición * 10 columnas
    nans = np.where(np.empty_like(np.arange((len(df_aux.index)*15-14)*10).reshape(len(df_aux.index)*15-14, 10)), np.nan, np.nan)
    df_new = pd.DataFrame(nans, columns=['ID','oldID','fecha','intensidad','ocupacion','vmed','periodo_integracion', 
                                         'KM', 'densidad', 'Hora'])
    j = 0
    for i in df_new.index.values.astype(int):
        if (i%15) == 0:
            if i != 0:
                j=j+1
            df_new.loc[i] = df_aux.values[j]
        else:
            df_new.iloc[i, df.columns.get_loc('ID')] = df_aux.iloc[j, df.columns.get_loc('ID')]
            df_new.iloc[i, df.columns.get_loc('oldID')] = df_aux.iloc[j, df.columns.get_loc('oldID')]
            df_new.iloc[i, df.columns.get_loc('fecha')] = df_aux.iloc[j, df.columns.get_loc('fecha')]
            df_new.iloc[i, df.columns.get_loc('periodo_integracion')] = 1.0
            df_new.iloc[i, df.columns.get_loc('KM')] = df_aux.iloc[j, df.columns.get_loc('KM')]
    
    
    df_new['intensidad'] = df_new['intensidad'].interpolate()
    df_new['ocupacion'] = df_new['ocupacion'].interpolate()
    df_new['vmed'] = df_new['vmed'].interpolate()
    df_new['densidad'] = df_new['densidad'].interpolate()
    #df_int['Hora'] = df_new['Hora'].interpolate()
    if id == 'PM10001':
        df_temp = df_new
    else:
        df_temp = df_temp.append(df_new, ignore_index=True)

oldID:  PM10001
oldID:  PM10141
oldID:  PM10211
oldID:  PM10241
oldID:  PM10341
oldID:  PM10401
oldID:  PM10441
oldID:  PM10471
oldID:  PM10501
oldID:  PM10561
oldID:  PM10901
oldID:  PM10941
oldID:  PM10981
oldID:  PM11071
oldID:  PM11101
oldID:  PM11161
oldID:  PM11201
oldID:  PM11301
oldID:  PM11981
oldID:  PM12061
oldID:  PM12121
oldID:  PM12211
oldID:  PM12241
oldID:  PM12331
oldID:  PM12391
oldID:  PM12471
oldID:  PM12571
oldID:  PM12641
oldID:  PM12721
oldID:  PM12781
oldID:  PM12851
oldID:  PM12921
oldID:  PM12961
oldID:  PM12981
oldID:  PM13011
oldID:  PM13041
oldID:  PM13101
oldID:  PM13181
oldID:  PM13211
oldID:  PM13221


In [3]:
df_temp

,ID,oldID,fecha,intensidad,ocupacion,vmed,periodo_integracion,KM,densidad,Hora
0,6639.0,PM10001,2015-08-04 00:00,848.000000,3.000000,77.000000,15.0,0.0,11.012987,0.00
1,6639.0,PM10001,2015-08-04 00:00,845.000000,3.000000,77.600000,1.0,0.0,10.901268,NaN
2,6639.0,PM10001,2015-08-04 00:00,842.000000,3.000000,78.200000,1.0,0.0,10.789550,NaN
3,6639.0,PM10001,2015-08-04 00:00,839.000000,3.000000,78.800000,1.0,0.0,10.677831,NaN
4,6639.0,PM10001,2015-08-04 00:00,836.000000,3.000000,79.400000,1.0,0.0,10.566113,NaN
5,6639.0,PM10001,2015-08-04 00:00,833.000000,3.000000,80.000000,1.0,0.0,10.454394,NaN
6,6639.0,PM10001,2015-08-04 00:00,830.000000,3.000000,80.600000,1.0,0.0,10.342676,NaN
7,6639.0,PM10001,2015-08-04 00:00,827.000000,3.000000,81.200000,1.0,0.0,10.230957,NaN
8,6639.0,PM10001,2015-08-04 00:00,824.000000,3.000000,81.800000,1.0,0.0,10.119239,NaN
9,6639.0,PM10001,2015-08-04 00:00,821.000000,3.000000,82.400000,1.0,0.0,10.007520,NaN


In [ ]:
# Exporto a CSV el DataFrame

df_good.to_csv('df_M30_KM_dia_inter_temp.csv', sep=';')

In [ ]:
df_good = pd.read_csv('df_M30_KM_dia_inter_temp.csv', delimiter=';')

In [ ]:
df_esp = pd.DataFrame([])
df_int = pd.DataFrame([])

df_good.Hora.round(2)

for time in np.sort(df_good.Hora.value_counts().keys()):
    print("Hora:", time)
    df_aux = df_good.loc[df_good['Hora'] == time]
    df_aux = df_aux.sort_values(by=['KM'])
    df_aux = df_aux.reset_index(drop=True)
    df_aux = df_aux.drop(columns=['Unnamed: 0'])
    
    nans = np.where(np.empty_like(np.arange(325*10).reshape(325, 10)), np.nan, np.nan)
    df_new = pd.DataFrame(nans, columns=['ID','oldID','fecha','intensidad','ocupacion','vmed','periodo_integracion','KM','densidad','Hora'])
    
    df_new['ID'] = 'int_temp'
    df_new['oldID'] = 'int_temp'
    df_new['Hora'] = time
    df_new['fecha'] = df_aux.iloc[0, df_aux.columns.get_loc('fecha')]
    df_new['periodo_integracion'] = 1.0
    
    #for km in df_aux['KM'].diff():
        #print("KM:", km)
    #    if not math.isnan(km):
            #print("km:", round(km, 1), "Size:", int(round((km/0.1)*10,1)), "Shape:", int(round(km/0.1,1)))
    #        nans = np.where(np.empty_like(np.arange(int(round((km/0.1)*10, 1))).reshape(int(round((km/0.1), 1)), 10)), np.nan, np.nan)
    #        df_new = pd.DataFrame(nans, columns=['ID','oldID','fecha','intensidad','ocupacion','vmed','periodo_integracion','KM','densidad','Hora'])
            
    #        df_new['ID'] = 'int_temp'
    #        df_new['oldID'] = 'int_temp'
    #        df_new['Hora'] = time
    #        df_new['fecha'] = df_aux.iloc[0, df_aux.columns.get_loc('fecha')]
    #        df_new['periodo_integracion'] = 1.0
            
    #        df_int = df_int.append(df_new, ignore_index=True)
    
    #print("Assigning values")
    j=0
    k = 0
    for i in df_new.index.values.astype(int):
        print("i:", i, "KM:", int(round(np.sort(df_good.KM.value_counts().keys())[k]/0.1, 1)))
        if i == int(round(np.sort(df_good.KM.value_counts().keys())[k]/0.1, 1)):
            print("Asigned")
            df_new.loc[i] = df_aux.loc[j].values
            j += 1
            k += 1
            
    print("Interpolating")
    #df_int_esp = df_int.interpolate()
    #if id == 'PM10001':
        #df_esp = df_int_esp
    #else:
        #df_esp = df_esp.append(df_int_esp, ignore_index=True)

In [ ]:
df_good

In [ ]:
df_esp.plot.scatter(x="Hora", y="densidad")
df_esp.plot.scatter(x="Hora", y="KM", c="vmed", colormap='viridis')